A first look at the data and eager to submit a notebook with my first insight.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

%matplotlib inline
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv("../input/train.csv")

print('Shape of train_df: ', train_df.shape)
train_df.head(10)

What is a vendor?    There are only two values for vendor_id 1 and 2 with roughly the same number of counts. 

In [ ]:
train_df['vendor_id'].value_counts()

## Translate pickup and dropoff coordinates into a distance

In [ ]:
import math
def Haversine(x):
    lat1 = x['pickup_latitude']
    lat2 = x['dropoff_latitude']
    lon1 = x['pickup_longitude']
    lon2 = x['dropoff_longitude']
    R = 6371000. #meters
    conversion=57.29575
    phi1 = lat1/conversion
    phi2 = lat2/conversion
    deltaPhi = phi1-phi2
    deltaLamb=(lon2-lon1)/conversion
    a=math.sin(deltaPhi/2.)*math.sin(deltaPhi/2.)+math.cos(phi1)*math.cos(phi2)*math.sin(deltaLamb/2.)*math.sin(deltaLamb/2.)
    c=2.*math.atan(math.sqrt(a)*math.sqrt(1.-a))
    return R*c/1000. #kilometers

In [ ]:
train_df['Geo_distance'] = train_df.apply(Haversine, axis=1)

In [ ]:
train_df.head()

Lets correlate the trip duration to the geodesic distance:

In [ ]:
y = np.log(train_df['Geo_distance'] , dtype='float64')
x = np.log(train_df['trip_duration'], dtype='float64')

plt.scatter(x, y, alpha=0.5)
plt.title('Scatter plot distance vs trip duration')
plt.xlabel('Log of trip duration [seconds]')
plt.ylabel('Geodesic distance [kilometers]')
plt.show()

Lets ignore the warning and come back to it latter.

From the above plot one can see some outliers; four trips with trip_duration longer than 400 hours and ~ 11 instances with geodesic distances of more than 200 kilometers.

Back to linear scales and changing seconds to hours we see below the first group of outliers:

In [ ]:
plt.scatter(train_df['trip_duration']/3600., train_df['Geo_distance'], alpha=0.5)
plt.title('Scatter plot distance vs trip duration')
plt.xlabel('trip duration [hours]')
plt.ylabel('Geodesic distance [kilometers]')
plt.show()

In [ ]:
train_df[(train_df['trip_duration']> 400.*3600.)]

I would guess that the driver did not disengage the meter and the car was not driven for many days.

Next we look at trips of more than 200 kilometers. This time, there could be an operator error of a possible failure in the Haversine distance I use. Looking at those 11 instances can help:

In [ ]:
train_df[(train_df['Geo_distance']> 200.)]

The first instance had its pickup somewhere in Quebec Canada and dropoff at 6 Av/w 35 St, New York, NY 10001, USA
I would also drop these instances. This looks like an error in the coordinate setup.  

In [ ]:
#just drop four entries with long trip durations and eleven with wrong coordinates (these sets do not overlap)

plt.scatter(train_df[(train_df['trip_duration']< 400.*3600.) & (train_df['Geo_distance']< 200.)].trip_duration/3600., 
            train_df[(train_df['trip_duration']< 400.*3600.) & (train_df['Geo_distance']< 200.)].Geo_distance, alpha=0.5)
plt.title('Scatter plot distance vs trip duration')
plt.xlabel('trip duration [hours]')
plt.ylabel('Geodesic distance [kilometers]')
plt.show()

Repeat the plots for each vendor_id

In [ ]:
#just drop four entries with long trip durations and eleven with wrong coordinates (these sets do not overlap)

plt.scatter(train_df[(train_df['trip_duration']< 400.*3600.) & (train_df['Geo_distance']< 200.)&(train_df['vendor_id']==1)].trip_duration/3600., 
            train_df[(train_df['trip_duration']< 400.*3600.) & (train_df['Geo_distance']< 200.)&(train_df['vendor_id']==1)].Geo_distance, alpha=0.5)
plt.title('Scatter plot distance vs trip duration VENDOR_ID = 1 ')
plt.xlabel('trip duration [hours]')
plt.ylabel('Geodesic distance [kilometers]')
plt.show()

In [ ]:
#just drop four entries with long trip durations and eleven with wrong coordinates (these sets do not overlap)

plt.scatter(train_df[(train_df['trip_duration']< 400.*3600.) & (train_df['Geo_distance']< 200.)&(train_df['vendor_id']==2)].trip_duration/3600., 
            train_df[(train_df['trip_duration']< 400.*3600.) & (train_df['Geo_distance']< 200.)&(train_df['vendor_id']==2)].Geo_distance, alpha=0.5)
plt.title('Scatter plot distance vs trip duration VENDOR_ID = 2 ')
plt.xlabel('trip duration [hours]')
plt.ylabel('Geodesic distance [kilometers]')
plt.show()

So it appears that vendor_id = 1 does short trips with a few outliers. These look more like the yellow taxis of NYC.
In contrast, the vendor_id = 2 trips have too blobs in duration and leave me all confused. How can trips that last longer be limited to ~ 20 km.

Now that I have submitted my first look I will read the discussions.